Conteo de palabras en Apache Pig
===

* Última modificación: Noviembre 05, 2019

Archivos de prueba
---

A continuación se generarán tres archivos de prueba para probar el sistema. Puede usar directamente comandos del sistema operativo en el Terminal y el editor de texto `pico` para crear los archivos.

In [1]:
#
# Se crea el directorio de entrada
#
!rm -rf input output
!mkdir input

In [2]:
%%writefile input/text0.txt
Analytics is the discovery, interpretation, and communication of meaningful patterns 
in data. Especially valuable in areas rich with recorded information, analytics relies 
on the simultaneous application of statistics, computer programming and operations research 
to quantify performance.

Organizations may apply analytics to business data to describe, predict, and improve business 
performance. Specifically, areas within analytics include predictive analytics, prescriptive 
analytics, enterprise decision management, descriptive analytics, cognitive analytics, Big 
Data Analytics, retail analytics, store assortment and stock-keeping unit optimization, 
marketing optimization and marketing mix modeling, web analytics, call analytics, speech 
analytics, sales force sizing and optimization, price and promotion modeling, predictive 
science, credit risk analysis, and fraud analytics. Since analytics can require extensive 
computation (see big data), the algorithms and software used for analytics harness the most 
current methods in computer science, statistics, and mathematics.

Writing input/text0.txt


In [3]:
%%writefile input/text1.txt
The field of data analysis. Analytics often involves studying past historical data to 
research potential trends, to analyze the effects of certain decisions or events, or to 
evaluate the performance of a given tool or scenario. The goal of analytics is to improve 
the business by gaining knowledge which can be used to make improvements or changes.

Writing input/text1.txt


In [4]:
%%writefile input/text2.txt
Data analytics (DA) is the process of examining data sets in order to draw conclusions 
about the information they contain, increasingly with the aid of specialized systems 
and software. Data analytics technologies and techniques are widely used in commercial 
industries to enable organizations to make more-informed business decisions and by 
scientists and researchers to verify or disprove scientific models, theories and 
hypotheses.

Writing input/text2.txt


Creación del contendor
---

En el Terminal ejecute el siguiente comando:

```bash
$ docker run --rm -it -v "$PWD":/workspace  --name pig -p 50070:50070 -p 8088:8088 -p 8888:8888  jdvelasq/pig:0.17.0
```

Como resultado debe ver el siguiente mensaje:

```
======================================

Hadoop NameNode at:

    http://127.0.0.1:50070/

Yarn ResourceManager at:

     http://127.0.0.1:8088/

======================================
```

Verificación de que Hadoop está ejecutándose correctamente
--

Para verificar que Hadoop se está ejecutando correctamente abra las siguiente direcciones en un explorador de internet:

* http://127.0.0.1:50070/


* http://127.0.0.1:8088/


Código en Apache Pig
---

**Nota.** Se usan los dos guiones `--` para comentario de una línea y `/*` ... `*/` para comentarios de varias líneas.

In [5]:
%%writefile script.pig

-- crea la carpeta input in el HDFS
fs -mkdir input

-- copia los archivos del sistema local al HDFS
fs -put input/ .

-- carga de datos
lines = LOAD 'input/text*.txt' AS (line:CHARARRAY);

-- genera una tabla llamada words con una palabra por registro
words = FOREACH lines GENERATE FLATTEN(TOKENIZE(line)) AS word;

-- agrupa los registros que tienen la misma palabra
grouped = GROUP words BY word;

-- genera una variable que cuenta las ocurrencias por cada grupo
wordcount = FOREACH grouped GENERATE group, COUNT(words);

-- selecciona las primeras 15 palabras
s = LIMIT wordcount 15;

-- escribe el archivo de salida 
STORE s INTO 'output';

-- copia los archivos del HDFS al sistema local
fs -get output/ .


Overwriting script.pig


In [6]:
!pig -execute 'run script.pig'

2022-05-12 19:49:21,209 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-05-12 19:49:23,182 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.textoutputformat.separator is deprecated. Instead, use mapreduce.output.textoutputformat.separator
2022-05-12 19:49:23,394 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:49:23,494 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2022-05-12 19:49:23,496 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2022-05-12 19:49:23,503 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use

Visualización de los resultados en el HDFS
---

In [7]:
!hadoop fs -ls output/*

-rw-r--r--   1 root supergroup          0 2022-05-12 19:50 output/_SUCCESS
-rw-r--r--   1 root supergroup         81 2022-05-12 19:50 output/part-r-00000


In [8]:
!hadoop fs -cat output/part-r-00000

a	1
DA	1
be	1
by	2
in	5
is	3
of	8
on	1
or	5
to	12
Big	1
The	2
aid	1
and	15
are	1


Movimiento de los resultados al sistema local
---

In [9]:
!hadoop fs -copyToLocal output output
!ls output/*

output/_SUCCESS  output/part-r-00000

output/output:
_SUCCESS  part-r-00000


Visualilzación de resultados en el sistema local
---

In [10]:
!ls -1 output/*

output/_SUCCESS
output/part-r-00000

output/output:
_SUCCESS
part-r-00000


In [11]:
!cat output/part-r-00000

a	1
DA	1
be	1
by	2
in	5
is	3
of	8
on	1
or	5
to	12
Big	1
The	2
aid	1
and	15
are	1


Notas
--

**Ejecución de programas en Grunt**

Se realiza con los comandos `exec` y `run`. 

    grunt> exec script
    
    grunt> run script
    
La diferencia entre estos comandos es que `exec` ejecuta el script sin importalo a `grunt`  mientras que `run` si lo hace.

**Ejecución de Pig en la consola de comandos**

Pig es comúnmente usado en la línea de comandos para ejecutar tareas de ETL, análisis de datos y procesamiento interactivo. Para ejecutar Pig en la línea de comandos digite:

    pig -x local
    
Si no usa la opción `x -local`, Pig usará el sistema de archivos HDFS.


La opción `-e` o `-execute` permite ejecutar un comandos simple sin entrar a Grunt:

    pig -e comando
    
    

**Ejecución de comandos del sistema operativo desde Pig**

`Pig` permite la ejecución de comandos del sistema operativo; por ejemplo:

     grunt> ls
     
También es posible usar comandos del sistema HDFS; el comando `hadoop dfs -ls /` se escribiría en `hive` como

     grunt> fs -ls / ;
     

#### Supresion de información detallada

Apache Pig imprime mucha información en pantalla relacionada con su ejecución y la Hadoop. Para regular el nivel de información entregada, se puede realizar una copia al directorio actual del archivo `./conf/log4j.properties` ubicado en la carpeta de instalación de Pig. 
    
El archivo `log4j.properties` se modifica para que se impriman únicamente los mensajes de error de Pig y Hadoop. Para ello, se modifica la línea correspondiente para que quede así:

    log4j.logger.org.apache.pig=error, A
    
y se agrega la siguiente para modificar el nivel de información entregado por Hadoop:

    log4j.logger.org.apache.hadoop=error, A
    
Se invoca Pig con:

    pig -4 log4j.properties
    
para indicar que se usa el archivo ubicado en la carpeta actual.

### Ejecución de Pig en Jupyter

A continuación se describe como ejecutar comandos de Pig en Jupyter usando la extensión de Jupyter `bigdata`. 

In [12]:
%load_ext bigdata

In [13]:
%timeout 300

In [14]:
%pig_start

#### WordCount en Apache Pig

Se cargan los archivos en Apache Pig.

In [15]:
%%pig
lines = LOAD 'input/text*.txt' AS (line:CHARARRAY);
DUMP lines;

 lines = LOAD 'input/text*.txt' AS (line:CHARARRAY);
 DUMP lines;
2022-05-12 19:50:14,604 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:50:14,720 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.reduce.markreset.buffer.percent
2022-05-12 19:50:14,722 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.output.compress is deprecated. Instead, use mapreduce.output.fileoutputformat.compress
2022-05-12 19:50:14,728 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.submit.replication is deprecated. Instead, use mapreduce.client.submit.file.replication
2022-05-12 19:50:14,891 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker.http.address is deprecated. Instead, use mapreduce.jobtracker.http.address
2022-05-12 19:50:14,894 [JobControl] INFO  org.apache.hadoop.y

Realiza el conteo de palabras.

In [16]:
%%pig
-- genera una tabla llamada words con una palabra por registro
words = FOREACH lines GENERATE FLATTEN(TOKENIZE(line)) AS word;
DUMP words;

 -- genera una tabla llamada words con una palabra por registro
 words = FOREACH lines GENERATE FLATTEN(TOKENIZE(line)) AS word;
 DUMP words;
2022-05-12 19:50:27,234 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:50:27,769 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:50:27,780 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 19:50:27,789 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 3
2022-05-12 19:50:27,814 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 19:50:27,843 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652384900661_0004
2022-05-12 19:50:27,845 [JobControl] INFO  org.apache.

In [17]:
%%pig
-- agrupa los registros que tienen la misma palabra
grouped = GROUP words BY word;
DUMP grouped;

 -- agrupa los registros que tienen la misma palabra
 grouped = GROUP words BY word;
 DUMP grouped;
2022-05-12 19:50:43,357 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:50:44,687 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:50:44,697 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 19:50:44,704 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 3
2022-05-12 19:50:45,131 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 19:50:45,552 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652384900661_0005
2022-05-12 19:50:45,554 [JobControl] INFO  org.apache.hadoop.mapred.YARNRunner - Job jar is not 

In [18]:
%%pig
-- genera una variable que cuenta las ocurrencias por cada grupo
wordcount = FOREACH grouped GENERATE group, COUNT(words);
DUMP wordcount;

 -- genera una variable que cuenta las ocurrencias por cada grupo
 wordcount = FOREACH grouped GENERATE group, COUNT(words);
 DUMP wordcount;
2022-05-12 19:51:01,164 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:51:02,456 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:51:02,468 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 19:51:02,476 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 3
2022-05-12 19:51:02,902 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 19:51:03,296 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652384900661_0006
2022-05-12 19:51:03,301 [JobControl] INFO  org.apache.

In [19]:
%%pig
-- selecciona las primeras 15 palabras
s = LIMIT wordcount 15;
DUMP s;

 -- selecciona las primeras 15 palabras
 s = LIMIT wordcount 15;
 DUMP s;
2022-05-12 19:51:23,855 [main] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:51:25,146 [JobControl] INFO  org.apache.hadoop.yarn.client.RMProxy - Connecting to ResourceManager at /0.0.0.0:8032
2022-05-12 19:51:25,156 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-05-12 19:51:25,164 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 3
2022-05-12 19:51:26,001 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-05-12 19:51:26,428 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_1652384900661_0007
2022-05-12 19:51:26,432 [JobControl] INFO  org.apache.hadoop.mapred.YARNRunner - Job jar is not present. Not adding any ja

In [20]:
%pig_quit

### Limpieza del HDFS y de la máquina local

In [21]:
## Se elimina el directorio de salida en el hdfs si existe
!hadoop fs -rm input/*
!hadoop fs -rm output/*
!hadoop fs -rmdir input output

Deleted input/text0.txt
Deleted input/text1.txt
Deleted input/text2.txt
Deleted output/_SUCCESS
rm: `output/output': No such file or directory
Deleted output/part-r-00000


In [22]:
!rm -rf input output